In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import os
import tarfile
from six.moves import urllib
import seaborn as sns
%matplotlib inline

In [2]:
DOWNLOAD_ROOT = 'https://raw.githubusercontent.com/ageron/handson-ml/master/'
HOUSING_PATH = 'datasets/housing'
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + '/housing.tgz'

In [3]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path= HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path = housing_path)
    housing_tgz.close()

In [4]:
def load_housing_data(housing_path = HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

In [5]:
fetch_housing_data()

In [6]:
housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


## SkLearn useful features !!!!
--------------------------------------------

In [22]:
from sklearn.model_selection import StratifiedShuffleSplit

In [23]:
split = StratifiedShuffleSplit(n_splits=1 , test_size=0.2, random_state=42)

In [24]:
for train_index , test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    

In [28]:
strat_train_set['income_cat'] = np.ceil(strat_train_set['median_income']/1.5)
strat_train_set['income_cat'].where(strat_train_set['income_cat']< 5, 5, inplace=True)

In [29]:
strat_train_set['income_cat'].value_counts() / len(strat_train_set)

3.0    0.350594
2.0    0.318859
4.0    0.176296
5.0    0.114402
1.0    0.039850
Name: income_cat, dtype: float64

In [30]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis = 1, inplace=True)

In [31]:
housingcopy = strat_train_set.copy()

In [32]:
housingcopy.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
17606,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042,286600.0,<1H OCEAN
18632,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214,340600.0,<1H OCEAN
14650,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621,196900.0,NEAR OCEAN
3230,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839,46300.0,INLAND
3555,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347,254500.0,<1H OCEAN


In [41]:
housingMl = housingcopy.drop('median_house_value', axis=1)

In [42]:
housing_labels = housingcopy['median_house_value'].copy()

In [43]:
housingMl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 17606 to 15775
Data columns (total 9 columns):
longitude             16512 non-null float64
latitude              16512 non-null float64
housing_median_age    16512 non-null float64
total_rooms           16512 non-null float64
total_bedrooms        16354 non-null float64
population            16512 non-null float64
households            16512 non-null float64
median_income         16512 non-null float64
ocean_proximity       16512 non-null object
dtypes: float64(8), object(1)
memory usage: 1.9+ MB


In [44]:
from sklearn.preprocessing import Imputer

### Imputer Parameters
-----------------------------

* missing_values : integer or “NaN”, optional (default=”NaN”). For missing values encoded as np.nan, use the string value “NaN”.
* strategy : string, optional (default=”mean”) --> The imputation strategy.
    1. If “mean”, then replace missing values using the mean along the axis.
    2. If “median”, then replace missing values using the median along the axis.
    3. If “most_frequent”, then replace missing using the most frequent value along the axis.
* axis : integer, optional (default=0) --> The axis along which to impute.
    1. If axis=0, then impute along columns.
    2. If axis=1, then impute along rows.
* verbose : integer, optional (default=0) --> Controls the verbosity of the imputer.
* copy : boolean, optional (default=True) If True, a copy of X will be created. If False, imputation will be done in-place whenever possible. 

In [45]:
imputer = Imputer(strategy = 'median')

In [46]:
housingMl_num = housingMl.drop('ocean_proximity',axis=1)

In [47]:
imputer.fit(housingMl_num)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

In [48]:
imputer.statistics_

array([ -118.51  ,    34.26  ,    29.    ,  2119.5   ,   433.    ,
        1164.    ,   408.    ,     3.5409])

In [49]:
X = imputer.transform(housingMl_num)

In [50]:
housingMl_num_tr = pd.DataFrame(X, columns=housingMl_num.columns)

In [51]:
housingMl_num_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 8 columns):
longitude             16512 non-null float64
latitude              16512 non-null float64
housing_median_age    16512 non-null float64
total_rooms           16512 non-null float64
total_bedrooms        16512 non-null float64
population            16512 non-null float64
households            16512 non-null float64
median_income         16512 non-null float64
dtypes: float64(8)
memory usage: 1.0 MB
